# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests
import math
from scipy import stats
import xlsxwriter

In [61]:
from scipy.stats import percentileofscore as score

In [62]:
from statistics import mean

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import API_TOKEN
from secrets import ALPHA_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [9]:
import datetime

In [36]:
def get_value(ticker, api_key):
    
    
    api_url = f'https://www.alphavantage.co/query?function=OVERVIEW&symbol={ticker}&apikey={api_key}'
    

    data = requests.get(api_url).json()
    market_cap = int(data['MarketCapitalization'])
    close_price = market_cap/int(data['SharesOutstanding'])
    return close_price

In [37]:
def get_one_year_return(ticker, api_key):

    base_url = "https://api.polygon.io/v2/aggs/ticker/"

    today = datetime.date.today().strftime("%Y-%m-%d")

    one_year_ago = (datetime.date.today() - datetime.timedelta(days=365)).strftime("%Y-%m-%d")

    

    url = f"{base_url}{ticker}/range/1/day/{one_year_ago}/{today}?adjusted=true&apiKey={api_key}"

    response = requests.get(url)
    

    data = response.json()['results']
    


    start_price = data[0]['c']

    end_price = data[-1]['c']

    

    return ((end_price - start_price) / start_price) * 100

In [38]:
def get_six_month_return(ticker, api_key):

    base_url = "https://api.polygon.io/v2/aggs/ticker/"

    today = datetime.date.today().strftime("%Y-%m-%d")

    one_year_ago = (datetime.date.today() - datetime.timedelta(days=180)).strftime("%Y-%m-%d")

    

    url = f"{base_url}{ticker}/range/1/day/{one_year_ago}/{today}?adjusted=true&apiKey={api_key}"

    response = requests.get(url)
    

    data = response.json()['results']
    


    start_price = data[0]['c']

    end_price = data[-1]['c']

    

    return ((end_price - start_price) / start_price) * 100

In [39]:
def get_three_month_return(ticker, api_key):

    base_url = "https://api.polygon.io/v2/aggs/ticker/"

    today = datetime.date.today().strftime("%Y-%m-%d")

    one_year_ago = (datetime.date.today() - datetime.timedelta(days=90)).strftime("%Y-%m-%d")

    

    url = f"{base_url}{ticker}/range/1/day/{one_year_ago}/{today}?adjusted=true&apiKey={api_key}"

    response = requests.get(url)
    

    data = response.json()['results']
    


    start_price = data[0]['c']

    end_price = data[-1]['c']

    

    return ((end_price - start_price) / start_price) * 100

In [40]:
def get_one_month_return(ticker, api_key):

    base_url = "https://api.polygon.io/v2/aggs/ticker/"

    today = datetime.date.today().strftime("%Y-%m-%d")

    one_year_ago = (datetime.date.today() - datetime.timedelta(days=30)).strftime("%Y-%m-%d")

    

    url = f"{base_url}{ticker}/range/1/day/{one_year_ago}/{today}?adjusted=true&apiKey={api_key}"

    response = requests.get(url)
    

    data = response.json()['results']
    


    start_price = data[0]['c']

    end_price = data[-1]['c']

    

    return ((end_price - start_price) / start_price) * 100

In [41]:
stocks = stocks.iloc[0:25,0:1]
import time
    

In [54]:
my_columns = ['Ticker','Stock Price','One-Year Price Return','One-Year Return Percentile','Six-Month Price Return','Six-Month Return Percentile','Three-Month Price Return','Three-Month Return Percentile','One-Month Price Return','One-Month Return Percentile','Number of Shares to Buy']

final_df = pd.DataFrame(columns = my_columns)
final_df

,Ticker,Stock Price,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,Number of Shares to Buy


In [55]:
for i in range(1,11):
    stock = stocks.loc[i-1].values[0]
    oneyear_return = get_one_year_return(stock,API_TOKEN)
    sixmonth_return = get_six_month_return(stock,API_TOKEN)
    threemonth_return = get_three_month_return(stock,API_TOKEN)
    onemonth_return = get_one_month_return(stock,API_TOKEN)
    value = get_value(stock,ALPHA_TOKEN)
    final_df = final_df.append(
            pd.Series(
            [
                stock,
                value,
                oneyear_return,
                'N/A',
                sixmonth_return,
                'N/A',
                threemonth_return,
                'N/A',
                onemonth_return,
                'N/A',
                'N/A',
                
            ],
            index = my_columns    
            ),

            ignore_index=True
        )
    
    
    time.sleep(80)
final_df
    
        

KeyError: 'results'

In [56]:
score = ['N/A','N/A','N/A','N/A','N/A']
final_df['HQM Score'] = score


In [59]:
final_df

,Ticker,Stock Price,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,Number of Shares to Buy,HQM Score
0,A,134.369992,7.745971,N/A,3.036577,N/A,-5.725111,N/A,2.151437,N/A,N/A,N/A
1,AAL,14.640000,20.295809,N/A,27.304348,N/A,40.904716,N/A,7.647059,N/A,N/A,N/A
2,AAP,43.840359,-15.104570,N/A,-37.938845,N/A,-5.005417,N/A,18.198975,N/A,N/A,N/A
3,AAPL,235.059984,23.455882,N/A,22.267880,N/A,2.293398,N/A,0.711225,N/A,N/A,N/A
4,ABBV,181.140004,31.184820,N/A,12.341851,N/A,-7.193360,N/A,-4.502320,N/A,N/A,N/A


## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [63]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]
for row in final_df.index:
    for time_period in time_periods:
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'
        final_df.loc[row, percentile_col] = score(final_df[change_col],final_df.loc[row,change_col])
final_df

,Ticker,Stock Price,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,Number of Shares to Buy,HQM Score
0,A,134.369992,7.745971,40.0,3.036577,40.0,-5.725111,40.0,2.151437,60.0,N/A,N/A
1,AAL,14.640000,20.295809,60.0,27.304348,100.0,40.904716,100.0,7.647059,80.0,N/A,N/A
2,AAP,43.840359,-15.104570,20.0,-37.938845,20.0,-5.005417,60.0,18.198975,100.0,N/A,N/A
3,AAPL,235.059984,23.455882,80.0,22.267880,80.0,2.293398,80.0,0.711225,40.0,N/A,N/A
4,ABBV,181.140004,31.184820,100.0,12.341851,60.0,-7.193360,20.0,-4.502320,20.0,N/A,N/A


In [64]:
for row in final_df.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(final_df.loc[row,f'{time_period} Return Percentile'])
    final_df.loc[row,'HQM Score'] = mean(momentum_percentiles)
final_df

,Ticker,Stock Price,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,Number of Shares to Buy,HQM Score
0,A,134.369992,7.745971,40.0,3.036577,40.0,-5.725111,40.0,2.151437,60.0,N/A,45.0
1,AAL,14.640000,20.295809,60.0,27.304348,100.0,40.904716,100.0,7.647059,80.0,N/A,85.0
2,AAP,43.840359,-15.104570,20.0,-37.938845,20.0,-5.005417,60.0,18.198975,100.0,N/A,50.0
3,AAPL,235.059984,23.455882,80.0,22.267880,80.0,2.293398,80.0,0.711225,40.0,N/A,70.0
4,ABBV,181.140004,31.184820,100.0,12.341851,60.0,-7.193360,20.0,-4.502320,20.0,N/A,50.0


In [67]:
final_df.sort_values('HQM Score', ascending=False,inplace=True)
final_df

,Ticker,Stock Price,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,Number of Shares to Buy,HQM Score
1,AAL,14.640000,20.295809,60.0,27.304348,100.0,40.904716,100.0,7.647059,80.0,N/A,85.0
3,AAPL,235.059984,23.455882,80.0,22.267880,80.0,2.293398,80.0,0.711225,40.0,N/A,70.0
2,AAP,43.840359,-15.104570,20.0,-37.938845,20.0,-5.005417,60.0,18.198975,100.0,N/A,50.0
4,ABBV,181.140004,31.184820,100.0,12.341851,60.0,-7.193360,20.0,-4.502320,20.0,N/A,50.0
0,A,134.369992,7.745971,40.0,3.036577,40.0,-5.725111,40.0,2.151437,60.0,N/A,45.0


## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [ ]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [68]:
position_size = 5000/len(final_df.index)
for i in range(0,len(final_df.index)):
    final_df.loc[i,'Number of Shares to Buy'] = math.floor(position_size/final_df.loc[i,'Stock Price'])
final_df

,Ticker,Stock Price,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,Number of Shares to Buy,HQM Score
1,AAL,14.640000,20.295809,60.0,27.304348,100.0,40.904716,100.0,7.647059,80.0,68,85.0
3,AAPL,235.059984,23.455882,80.0,22.267880,80.0,2.293398,80.0,0.711225,40.0,4,70.0
2,AAP,43.840359,-15.104570,20.0,-37.938845,20.0,-5.005417,60.0,18.198975,100.0,22,50.0
4,ABBV,181.140004,31.184820,100.0,12.341851,60.0,-7.193360,20.0,-4.502320,20.0,5,50.0
0,A,134.369992,7.745971,40.0,3.036577,40.0,-5.725111,40.0,2.151437,60.0,7,45.0


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [70]:
# Best two Stocks
final_df = final_df[:2]
final_df

,Ticker,Stock Price,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,Number of Shares to Buy,HQM Score
1,AAL,14.640000,20.295809,60.0,27.304348,100.0,40.904716,100.0,7.647059,80.0,68,85.0
3,AAPL,235.059984,23.455882,80.0,22.267880,80.0,2.293398,80.0,0.711225,40.0,4,70.0


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [ ]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

## Saving Our Excel Output

As before, saving our Excel output is very easy: